In [1]:
import os
from tqdm.notebook import tqdm
import gc
from torch.nn import Parameter
import torch.nn.functional as F
import torch.nn as nn
import math
import timm
import pandas as pl
import torch
import numpy as np
from torch.amp import GradScaler
import cv2
import random
from tqdm import tqdm
from torch.autograd import Variable
from skimage.metrics import structural_similarity as ssim

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(228)

In [3]:
path2files = '/kaggle/input/ioai-contest-2/'

pairs = pl.read_csv(path2files + 'imgs/pairs_list.csv')
paths_embeds = pl.read_csv(path2files + 'imgs/paths_embeds.csv')['image_path']
real_embeds = np.load(path2files + 'imgs/real_embeds.npy')

In [4]:
class MCSDataset(torch.utils.data.Dataset):
    def __init__(self, image_path, target, imsize = 112):
        self.image_path = image_path
        self.target = target
        self.image_size = imsize

    def __len__(self):
        return len(self.target)

    def resize(self, img, interp):
        return  cv2.resize(
            img, (self.image_size, self.image_size), interpolation=interp)

    def __getitem__(self, idx):
        path = self.image_path[idx]
        target = self.target[idx]
        img = cv2.imread(f'{path2files}imgs/train/{path}')
        img = cv2.resize(
            img, (self.image_size, self.image_size), interpolation= cv2.INTER_LINEAR)

        img = (img / 255.) - 0.5
        img = np.transpose(img,(2,0,1)).astype(np.float32)
        img = torch.from_numpy(img)
        target = torch.from_numpy(target)

        return img, target

In [5]:
def make_predict(model, val_loader, val_target, loss_func, DEVICE = 'cuda'):
    preds = []
    model.eval()
    average_loss = 0
    with torch.no_grad():
        for batch_number,  (img, target) in enumerate(val_loader):
            img = img.to(DEVICE)
            target = target.to(DEVICE)

            with torch.amp.autocast('cuda'):
                outputs = model(img)
                loss = loss_func(outputs, target)

            average_loss += loss.cpu().detach().numpy()
            preds += [outputs.to('cpu').numpy()]
    preds = np.concatenate(preds)
    print('MSE: ', ((preds -  np.array(val_target)) ** 2).mean())

In [6]:
class Model(nn.Module):
    def __init__(self, model_name,):
        super().__init__()
        self.model_name = model_name
        self.timm_ = timm.create_model(model_name, global_pool='', num_classes=0, in_chans=3)
        output_features = self.timm_(torch.zeros((1, 3, 112, 112))).shape[1]

        # self.new_layer = nn.Sequential(
        #     nn.Linear(output_features, 512),
        #     nn.ReLU(),
        #     nn.Dropout(0.5)
        # )

        self.norm = nn.BatchNorm1d(512)
    def forward(self, x):
        out_ = self.timm_(x).mean(dim=(2, 3))
        # out_ = self.new_layer(out_)
        out_ = self.norm(out_)
        out_ = F.normalize(out_)
        return out_

In [7]:
gc.collect()
torch.cuda.empty_cache()

In [8]:
batch_size = 8
valid_batch_size = 64
epochs = 10
lr = 0.001
clip_grad_norm = 15.28
DEVICE = 'cuda'

params_train = {'batch_size': batch_size, 'shuffle': True, 'drop_last': True, 'num_workers': 6}
params_val = {'batch_size': batch_size, 'shuffle': False, 'drop_last': False, 'num_workers': 6}

In [9]:
train_path = [x for i, x in enumerate(paths_embeds) if i % 5 != 0 ]
train_target = [x for i, x in enumerate(real_embeds) if i % 5 != 0 ]


val_path = [x for i, x in enumerate(paths_embeds) if i % 5 == 0 ]
val_target = [x for i, x in enumerate(real_embeds) if i % 5 == 0 ]

In [10]:
train_loader = torch.utils.data.DataLoader(MCSDataset(train_path, train_target), **params_train)
val_loader = torch.utils.data.DataLoader(MCSDataset(val_path, val_target), **params_val)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [11]:
model = Model('resnet18').cuda()
num_train_steps = int(len(train_loader) / batch_size  * epochs)
loss_func = torch.nn.MSELoss()

In [12]:
scaler = GradScaler('cuda')
optimizer = torch.optim.AdamW(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader) * epochs, 0.0000001)

In [13]:
for epoch in range(epochs):
    model.train()
    average_loss = 0
    for batch_number,  (img, target) in enumerate(train_loader):
        optimizer.zero_grad()
        img = img.to(DEVICE)
        target = target.to(DEVICE)
        
        with torch.amp.autocast('cuda'):
            outputs = model(img)
            loss = loss_func(outputs, target)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        average_loss += loss.cpu().detach().numpy()
    make_predict(model, val_loader,val_target, loss_func)
    
std_m = model.state_dict()
torch.save(std_m, f'model_student.pt')

MSE:  0.0023780572
MSE:  0.0019935227
MSE:  0.001789554
MSE:  0.0016780333
MSE:  0.0016003742
MSE:  0.0015518958
MSE:  0.0015268045
MSE:  0.0015163604
MSE:  0.0015124416
MSE:  0.0015148617


In [14]:
def read_img(path, image_size=112):
    img = cv2.imread(f'{path2files}imgs/train/{path}')
    img_ = cv2.resize(
        img, (image_size, image_size), interpolation=cv2.INTER_LINEAR)
    img = (img_ / 255.) - 0.5
    img = np.transpose(img, (2, 0, 1)).astype(np.float32)
    img = torch.from_numpy(img)
    return img, img_

In [15]:
max_iter = 100
loss = nn.MSELoss()
eps = 1e-3
eps = 0.016
attacked_img_dict = {}


for sour, targ in tqdm(zip(pairs['source_imgs'], pairs['target_imgs'])):

    target_descriptors = np.ones((5, 512), dtype=np.float32)
    targ = targ.split('|')
    sour = sour.split('|')

    list_tagt_img = []
    for i, t in enumerate(targ):
        img, orig_tgt = read_img(t)
        list_tagt_img += [orig_tgt]
        img = img.unsqueeze(0).cuda(non_blocking = True)
        res = model(Variable(img, requires_grad=False)).data.cpu().numpy().squeeze()
        target_descriptors[i] = res

    for ii, s in enumerate(sour): 
        img, orig_img = read_img(s)
        img = img.unsqueeze(0).cuda(non_blocking = True)
        input_var  = Variable(img, requires_grad=True)
        attacked_img = orig_img
        for iter_number in (range(max_iter)):
            adv_noise = torch.zeros((3,112,112)).cuda(non_blocking = True)
            for tg in target_descriptors:
                target_out = Variable(torch.from_numpy(tg).unsqueeze(0).cuda(non_blocking=True), requires_grad=False)
                input_var.grad = None
                out = model(input_var)
                calc_loss = loss(out, target_out)
                calc_loss.backward()
                # noise = eps * torch.sign(input_var.grad.data)\
                #                     .squeeze()
                noise = eps * torch.clamp(input_var.grad.data / input_var.grad.data.std(), -2, 2)
                adv_noise = adv_noise + noise

            input_var.data = input_var.data - adv_noise

            changed_img = input_var.data.cpu().squeeze()
            changed_img = ((changed_img + 0.5) * 255)
            changed_img[changed_img < 0] = 0
            changed_img[changed_img > 255] = 255
            changed_img = np.transpose(changed_img.numpy(), (1, 2, 0)).astype(np.int16)
            ssim_score = ssim(orig_img, changed_img, channel_axis=2, data_range = 256)
            if ssim_score < 0.95:
                break
            else:
                attacked_img = changed_img
        attacked_img_dict[s] = attacked_img

1000it [04:10,  3.99it/s]


In [16]:
sample_submission = pl.read_csv(path2files + 'imgs/sample_submission.csv')
sample_submission_df = pl.DataFrame()
sample_submission_df['Id'] = sample_submission['Id']

result = []
for id_ in tqdm(sample_submission_df['Id']):
    result += ['|'.join([str(i) for i in attacked_img_dict[id_].flatten().tolist()])]
sample_submission_df['Target'] = result

100%|██████████| 5000/5000 [00:26<00:00, 186.11it/s]


In [17]:
sample_submission_df.to_csv('submission.csv', index=None)

In [18]:
#!kaggle competitions submit -c ioai-contest-2 -f sample_submission_fgvm_attack.csv